In [ ]:
# On combine les deux leçons précédentes : OpenAI fonctions et chaînes

In [ ]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [ ]:
from typing import List
from pydantic import BaseModel, Field

In [ ]:
# Pydantic Syntax

In [ ]:
# Manière plus élégante de définir des structures de classes en Python
# Utilisé at eruntime pour data validation et conversion
# La définition explicite des fonctions OpenAI est longue

In [1]:
# Standard Python
class User:
    def __init__(self, name: str, age: int, email: str):
        self.name = name
        self.age = age
        self.email = email

In [2]:
foo = User(name="Joe",age=32, email="joe@gmail.com")

In [3]:
foo.name

'Joe'

In [4]:
foo = User(name="Joe",age="bar", email="joe@gmail.com")

In [5]:
# Python accepte des attributs qui ne respectent pas le format initialement demandé, 
# c'est un problème pour faire tourner les chaînes
foo.age

'bar'

In [ ]:
# Idem avec Pydantic

class pUser(BaseModel):
    name: str
    age: int
    email: str

In [ ]:
foo_p = pUser(name="Jane", age=32, email="jane@gmail.com")

In [ ]:
foo_p.name

In [ ]:
# Avec Pydantic, l'instanciation sur de mauvais formats renvoie une erreur
foo_p = pUser(name="Jane", age="bar", email="jane@gmail.com")

In [ ]:
# On peut encapsuler plusieurs niveaux de classes
# On définit la classe Class en tant que liste de pUser (ex. un classe d'étudiants)
class Class(BaseModel):
    students: List[pUser]

In [ ]:
obj = Class(
    students=[pUser(name="Jane", age=32, email="jane@gmail.com")]
)

In [ ]:
obj

In [ ]:
# Fonction pour relier Pydantic à OpenAI

In [ ]:
# On crée une class qui va nous servir pour créer des JSON, et donc définir des fonctions
# Field est un module de Pydantic permettant d'intégrer des éléments, tels que "description" ici
class WeatherSearch(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str = Field(description="airport code to get weather for")

In [ ]:
# Module à invoquer pour convertir en fonction OpenAI
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

In [ ]:
# Ci-dessous, non seulement on instancie un objet de la classe WeatherSearch,
# mais en plus on le convertit en une fonction au format JSON
weather_function = convert_pydantic_to_openai_function(WeatherSearch)

In [ ]:
weather_function

In [ ]:
# Le string document qui permet de décrire la classe est indispensable,
# c'est ce qui sert de description générale à la future fonction
class WeatherSearch1(BaseModel):
    airport_code: str = Field(description="airport code to get weather for")

In [ ]:
# Ne marche pas
convert_pydantic_to_openai_function(WeatherSearch1)

In [ ]:
# Ici on ne décrit pas airport_code avec Field dans cette classe
# La fonction pourra être créée mais la future "property" airport_code n'aura pas de description
class WeatherSearch2(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str

In [ ]:
convert_pydantic_to_openai_function(WeatherSearch2)

In [ ]:
# On combine Pydantic et un modèle
from langchain.chat_models import ChatOpenAI

In [ ]:
model = ChatOpenAI()

In [ ]:
model.invoke("what is the weather in SF today?", functions=[weather_function])

In [ ]:
# Pour éviter d'appeler les fonctions en même temps qu'on invoque le modèle, on utilise bind
# pour intégrer la fonction au modèle
model_with_function = model.bind(functions=[weather_function])

In [ ]:
model_with_function.invoke("what is the weather in sf?")

In [ ]:
# Utilisation forcée d'une fonction

In [ ]:
model_with_forced_function = model.bind(functions=[weather_function], function_call={"name":"WeatherSearch"})

In [ ]:
model_with_forced_function.invoke("what is the weather in sf?")

In [ ]:
model_with_forced_function.invoke("hi!")

In [ ]:
# Utilisation d'une chaîne

In [ ]:
from langchain.prompts import ChatPromptTemplate

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("user", "{input}")
])

In [ ]:
chain = prompt | model_with_function

In [ ]:
chain.invoke({"input": "what is the weather in sf?"})

In [ ]:
# Multiple functions
# On laisse le LLM décider quelle fonction est la plus pertinente pour répondre à la question

In [ ]:
class ArtistSearch(BaseModel):
    """Call this to get the names of songs by a particular artist"""
    artist_name: str = Field(description="name of artist to look up")
    n: int = Field(description="number of results")

In [ ]:
functions = [
    convert_pydantic_to_openai_function(WeatherSearch),
    convert_pydantic_to_openai_function(ArtistSearch),
]

In [ ]:
model_with_functions = model.bind(functions=functions)

In [ ]:
model_with_functions.invoke("what is the weather in sf?")

In [ ]:
model_with_functions.invoke("what are three songs by taylor swift?")

In [ ]:
model_with_functions.invoke("hi!")